In [1]:
import pandas as pd
import numpy as np
import glob

import os
os.chdir("C:/Users/Ruben/Desktop/Python Repository")

##### Load Data

In [2]:
#load - MSB/Deposit SARs, all states 2020-2021 aggregated at the county / file type level
# the glob package imports multiple csv files (21 files in this instance) at once from a single folder
path = r"C:\Users\Ruben\Desktop\Python Repository\dep.msb.AL-WY.20-21.countysusp"
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, thousands=",")
    li.append(df)

data = pd.concat(li, axis=0, ignore_index=True)

In [3]:
data

,Year Month,State,Countym,Industry,Suspicious Activity,Count
0,2020 January,Alabama,"Autauga County, AL",Depository Institution,Check,3
1,2020 January,Alabama,"Autauga County, AL",Depository Institution,Counterfeit Instrument,2
2,2020 January,Alabama,"Autauga County, AL",Depository Institution,Credit/Debit Card,1
3,2020 January,Alabama,"Autauga County, AL",Depository Institution,Other Fraud (Type),1
4,2020 January,Alabama,"Autauga County, AL",Depository Institution,Provided Questionable or False Documentation,1
...,...,...,...,...,...,...
1550581,2021 December,Wyoming,"Washakie County, WY",Money Services Business (MSB),[Total],2
1550582,2021 December,Wyoming,[Total],Money Services Business (MSB),[Total],299
1550583,2021 December,[Total],[Total],Money Services Business (MSB),[Total],14494
1550584,2021 [Total],[Total],[Total],Money Services Business (MSB),[Total],151004


In [5]:
#delete the subtotal rows that contain "[Total]"
newdata1 = data[~data.isin(["[Total]"]).any(axis=1)]
newdata1
#newdata1.shape (1432673, 6)

(1432673, 6)

In [6]:
#splitting "Year Month" column, rejoining it back to df,  and then renaming the column
#this date format is for tableau efficiency
newdata1 =newdata1.join(newdata1["Year Month"].str.partition(" ")[[0,2]].rename({0:"Year",2:"Month"}, axis=1))
newdata1["Date"] = newdata1["Month"] +" "+ newdata1["Year"] 

#drop unnecessary columns
newdata1 = newdata1.drop(["Year Month", "Month"], axis=1)
newdata1

,State,Countym,Industry,Suspicious Activity,Count,Year,Date
0,Alabama,"Autauga County, AL",Depository Institution,Check,3,2020,January 2020
1,Alabama,"Autauga County, AL",Depository Institution,Counterfeit Instrument,2,2020,January 2020
2,Alabama,"Autauga County, AL",Depository Institution,Credit/Debit Card,1,2020,January 2020
3,Alabama,"Autauga County, AL",Depository Institution,Other Fraud (Type),1,2020,January 2020
4,Alabama,"Autauga County, AL",Depository Institution,Provided Questionable or False Documentation,1,2020,January 2020
...,...,...,...,...,...,...,...
1550575,Wyoming,"Uinta County, WY",Money Services Business (MSB),Transaction(s) Below BSA Recordkeeping Threshold,1,2021,December 2021
1550576,Wyoming,"Uinta County, WY",Money Services Business (MSB),Two or More Individuals Working Together,7,2021,December 2021
1550577,Wyoming,"Uinta County, WY",Money Services Business (MSB),Wire,1,2021,December 2021
1550579,Wyoming,"Washakie County, WY",Money Services Business (MSB),Transaction(s) Below BSA Recordkeeping Threshold,1,2021,December 2021


In [7]:
#find rows that have missing values
null_data = newdata1[newdata1.isnull().any(axis=1)]
null_data.head() 
#no missing data in the SAR dataset as seen below

,State,Countym,Industry,Suspicious Activity,Count,Year,Date


In [10]:
#rename column
newdata1.rename(columns={"Countym":"County"}, inplace=True) 

#changing this county name to match with the county name in the FIPS dataset
newdata1["County"].replace("DoÃ±a Ana County, NM","Dona Ana County, NM",inplace=True)
newdata1

,State,County,Industry,Suspicious Activity,Count,Year,Date
0,Alabama,"Autauga County, AL",Depository Institution,Check,3,2020,January 2020
1,Alabama,"Autauga County, AL",Depository Institution,Counterfeit Instrument,2,2020,January 2020
2,Alabama,"Autauga County, AL",Depository Institution,Credit/Debit Card,1,2020,January 2020
3,Alabama,"Autauga County, AL",Depository Institution,Other Fraud (Type),1,2020,January 2020
4,Alabama,"Autauga County, AL",Depository Institution,Provided Questionable or False Documentation,1,2020,January 2020
...,...,...,...,...,...,...,...
1550575,Wyoming,"Uinta County, WY",Money Services Business (MSB),Transaction(s) Below BSA Recordkeeping Threshold,1,2021,December 2021
1550576,Wyoming,"Uinta County, WY",Money Services Business (MSB),Two or More Individuals Working Together,7,2021,December 2021
1550577,Wyoming,"Uinta County, WY",Money Services Business (MSB),Wire,1,2021,December 2021
1550579,Wyoming,"Washakie County, WY",Money Services Business (MSB),Transaction(s) Below BSA Recordkeeping Threshold,1,2021,December 2021


##### SAR dataset (above) will be merged with the FIPS data set (below).
The FIPS county code is a five-digit Federal Information Processing Standards (FIPS) code which uniquely identifies counties and county equivalents in the United States. The FIPS code is needed to accurately visualize county data in Tableau. When merging the datasets the first time, some of the issues I encountered where misspellings (capital case, hyphens and word spaces), outdated fips codes and even non-existant counties in the FinCen SAR stat database. All spelling corrections were made in the FIPS dataset to preserve the integrity of the FinCen dataset as much as possible.

In [11]:
# importing the FIPS dataset
fips = pd.read_excel(r"C:\Users\Ruben\Desktop\Python Repository\US_FIPS2.xlsx")

#make second row into the header
new_header = fips.iloc[0]
fips= fips[1:]
fips.columns = new_header
 
fips
#fips.dtypes

,FIPS State,County,FIPS County Name,FIPS State Code,FIPS County Code
1,Alabama,"Autauga County, AL",Autauga,01,001
2,Alabama,"Baldwin County, AL",Baldwin,01,003
3,Alabama,"Barbour County, AL",Barbour,01,005
4,Alabama,"Bibb County, AL",Bibb,01,007
5,Alabama,"Blount County, AL",Blount,01,009
...,...,...,...,...,...
3123,Wyoming,"Sweetwater County, WY",Sweetwater,56,037
3124,Wyoming,"Teton County, WY",Teton,56,039
3125,Wyoming,"Uinta County, WY",Uinta,56,041
3126,Wyoming,"Washakie County, WY",Washakie,56,043


In [12]:
#merge dataframes
mrgdata = pd.merge(fips,newdata1, how= "left", left_on=["County"], right_on=["County"]) 
mrgdata 


,FIPS State,County,FIPS County Name,FIPS State Code,FIPS County Code,State,Industry,Suspicious Activity,Count,Year,Date
0,Alabama,"Autauga County, AL",Autauga,01,001,Alabama,Depository Institution,Check,3.0,2020,January 2020
1,Alabama,"Autauga County, AL",Autauga,01,001,Alabama,Depository Institution,Counterfeit Instrument,2.0,2020,January 2020
2,Alabama,"Autauga County, AL",Autauga,01,001,Alabama,Depository Institution,Credit/Debit Card,1.0,2020,January 2020
3,Alabama,"Autauga County, AL",Autauga,01,001,Alabama,Depository Institution,Other Fraud (Type),1.0,2020,January 2020
4,Alabama,"Autauga County, AL",Autauga,01,001,Alabama,Depository Institution,Provided Questionable or False Documentation,1.0,2020,January 2020
...,...,...,...,...,...,...,...,...,...,...,...
1432695,Wyoming,"Weston County, WY",Weston,56,045,Wyoming,Money Services Business (MSB),Other Fraud (Type),1.0,2020,December 2020
1432696,Wyoming,"Weston County, WY",Weston,56,045,Wyoming,Money Services Business (MSB),Suspicious use of multiple transaction locations,1.0,2021,June 2021
1432697,Wyoming,"Weston County, WY",Weston,56,045,Wyoming,Money Services Business (MSB),Transaction(s) Below BSA Recordkeeping Threshold,1.0,2021,June 2021
1432698,Wyoming,"Weston County, WY",Weston,56,045,Wyoming,Money Services Business (MSB),Suspicious use of multiple transaction locations,2.0,2021,October 2021


In [20]:
#find rows that have missing values
null_data25 = mrgdata[mrgdata.isnull().any(axis=1)]
null_data25
#null_data25.to_csv("null_data25.csv") exported to excel for more research
#the large majority of the counties with missing data are sparsely populated counties that do not have SARs for the years 2021-2021



,FIPS State,County,FIPS County Name,FIPS State Code,FIPS County Code,State,Industry,Suspicious Activity,Count,Year,Date
501032,Kentucky,"Robertson County, KY",Robertson,21,201,NaN,NaN,NaN,0,NaN,NaN
725586,Montana,"Carter County, MT",Carter,30,011,NaN,NaN,NaN,0,NaN,NaN
728371,Montana,"Garfield County, MT",Garfield,30,033,NaN,NaN,NaN,0,NaN,NaN
728467,Montana,"Golden Valley County, MT",Golden Valley,30,037,NaN,NaN,NaN,0,NaN,NaN
730073,Montana,"McCone County, MT",McCone,30,055,NaN,NaN,NaN,0,NaN,NaN
731239,Montana,"Petroleum County, MT",Petroleum,30,069,NaN,NaN,NaN,0,NaN,NaN
732928,Montana,"Treasure County, MT",Treasure,30,103,NaN,NaN,NaN,0,NaN,NaN
734475,Nebraska,"Arthur County, NE",Arthur,31,005,NaN,NaN,NaN,0,NaN,NaN
734476,Nebraska,"Banner County, NE",Banner,31,007,NaN,NaN,NaN,0,NaN,NaN
734477,Nebraska,"Blaine County, NE",Blaine,31,009,NaN,NaN,NaN,0,NaN,NaN


In [21]:
# the "State" column cannot have missing values or else the tableau map will look like an incomplete puzzle with missing pieces
# to solve this, I populated the empty values in the "State" column with values from the "FIPS State" column 
# the "FIPS State" column is an addditional column that was kept in the merge and not necessarily needed
mrgdata["State"] = mrgdata["State"].fillna(mrgdata.pop("FIPS State"))
mrgdata

,County,FIPS County Name,FIPS State Code,FIPS County Code,State,Industry,Suspicious Activity,Count,Year,Date
0,"Autauga County, AL",Autauga,01,001,Alabama,Depository Institution,Check,3,2020,January 2020
1,"Autauga County, AL",Autauga,01,001,Alabama,Depository Institution,Counterfeit Instrument,2,2020,January 2020
2,"Autauga County, AL",Autauga,01,001,Alabama,Depository Institution,Credit/Debit Card,1,2020,January 2020
3,"Autauga County, AL",Autauga,01,001,Alabama,Depository Institution,Other Fraud (Type),1,2020,January 2020
4,"Autauga County, AL",Autauga,01,001,Alabama,Depository Institution,Provided Questionable or False Documentation,1,2020,January 2020
...,...,...,...,...,...,...,...,...,...,...
1432695,"Weston County, WY",Weston,56,045,Wyoming,Money Services Business (MSB),Other Fraud (Type),1,2020,December 2020
1432696,"Weston County, WY",Weston,56,045,Wyoming,Money Services Business (MSB),Suspicious use of multiple transaction locations,1,2021,June 2021
1432697,"Weston County, WY",Weston,56,045,Wyoming,Money Services Business (MSB),Transaction(s) Below BSA Recordkeeping Threshold,1,2021,June 2021
1432698,"Weston County, WY",Weston,56,045,Wyoming,Money Services Business (MSB),Suspicious use of multiple transaction locations,2,2021,October 2021


In [22]:
#verify that "State" column has zero missing values
null_data26 = mrgdata[mrgdata.isnull().any(axis=1)]
null_data26

,County,FIPS County Name,FIPS State Code,FIPS County Code,State,Industry,Suspicious Activity,Count,Year,Date
501032,"Robertson County, KY",Robertson,21,201,Kentucky,NaN,NaN,0,NaN,NaN
725586,"Carter County, MT",Carter,30,011,Montana,NaN,NaN,0,NaN,NaN
728371,"Garfield County, MT",Garfield,30,033,Montana,NaN,NaN,0,NaN,NaN
728467,"Golden Valley County, MT",Golden Valley,30,037,Montana,NaN,NaN,0,NaN,NaN
730073,"McCone County, MT",McCone,30,055,Montana,NaN,NaN,0,NaN,NaN
731239,"Petroleum County, MT",Petroleum,30,069,Montana,NaN,NaN,0,NaN,NaN
732928,"Treasure County, MT",Treasure,30,103,Montana,NaN,NaN,0,NaN,NaN
734475,"Arthur County, NE",Arthur,31,005,Nebraska,NaN,NaN,0,NaN,NaN
734476,"Banner County, NE",Banner,31,007,Nebraska,NaN,NaN,0,NaN,NaN
734477,"Blaine County, NE",Blaine,31,009,Nebraska,NaN,NaN,0,NaN,NaN


In [24]:
#fill missing values with 0
mrgdata[["Count"]] = mrgdata[["Count"]].fillna(0)

#change data type back to integer
mrgdata["Count"] = mrgdata["Count"].astype(int)

#mrgdata.dtypes
#output as csv to be imported into tableau
mrgdata.to_csv("50states.20-21.countysuspv2.csv")